# Assignment # 3:  Invoke models using Bedrock APIs

## Miguel Herrera

The following assignment is composed of 2 parts:

Section 1) Access models using the **invoke_model()** function

Section 2) Access models using the **invoke_model_with_response_stream()** function

**Note:** Please expand all sections to see code for each model.
   

In [81]:
#https://python.langchain.com/docs/integrations/chat/bedrock/

%pip install -r requirements.txt -Uq

Note: you may need to restart the kernel to use updated packages.


In [122]:
import boto3
import json
session = boto3.Session()
bedrock = session.client(
    service_name='bedrock',
)

In [84]:
output_text = bedrock.list_foundation_models()

In [86]:
output_text['modelSummaries']

[{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
  'modelId': 'amazon.titan-tg1-large',
  'modelName': 'Titan Text Large',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}},
 {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-g1-text-02',
  'modelId': 'amazon.titan-embed-g1-text-02',
  'modelName': 'Titan Text Embeddings v2',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['EMBEDDING'],
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}},
 {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-lite-v1:0:4k',
  'modelId': 'amazon.titan-text-lite-v1:0:4k',
  'modelName': 'Titan Text G1 - Lite',
  'providerName': '

## ---------- Creating the Bedrock runtime client to be used by all functions below ----------------

In [123]:
# Setup bedrock runtime client for invoking models
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime"
)

# SECTION 1: Invoking Models (Invoke Model) 

## Antropic Claude

In [482]:
def model_invoke(prompt):


    modelId = "anthropic.claude-v2:1"
    
    #configuring the payload
    prompt_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 4096,
        "temperature":0.1,
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                ],
            }
        ],
    }

    body = json.dumps(prompt_config)

    #modelId = ["anthropic.claude-3-sonnet-20240229-v1:0","anthropic.claude-3-haiku-20240307-v1:0", "anthropic.claude-v2:1"]	
    accept = "application/json"
    contentType = "application/json"

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    results = response_body.get("content")[0].get("text")
    return results

In [483]:
text = "Amazon Bedrock is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI. Using Amazon Bedrock, you can easily experiment with and evaluate top FMs for your use case, privately customize them with your data using techniques such as fine-tuning and Retrieval Augmented Generation (RAG), and build agents that execute tasks using your enterprise systems and data sources. Since Amazon Bedrock is serverless, you don't have to manage any infrastructure, and you can securely integrate and deploy generative AI capabilities into your applications using the AWS services you are already familiar with"

In [484]:
# Summarization

prompt = "Can you give me 2 real life applications of RAG systems."

text = "Breakfast in bed is the right way to start the day!"
text = "Breakfast in the most important meal of the day!"

prompt = f"Summarize the following text in 50 words or less.  Keep in mind that the comment might be sarcastic in nature or not.  Also provide a score of how sarcastic the comment might be: {text}"

print(model_invoke(prompt))
print()
print()

text = "The sky on a sunny day is blue!"
prompt = f"Summarize the following text in 50 words or less.  Keep in mind that the comment might be sarcastic in nature or not.  Also provide a score of how sarcastic the comment might be: {text}"

print(model_invoke(prompt))

Summary: The text states that breakfast is the most important meal of the day, possibly in a sarcastic tone.

Sarcasm score: 6/10

The text is short and the claim about breakfast being the most important meal is commonly said sincerely. However, the exclamation point suggests it could be meant sarcastically. Without more context, it's hard to tell definitively.


Here is a 47-word summary of the text:

The text states that the sky on a sunny day is blue. There is not enough context to determine if this comment is intended to be sarcastic. I would rate the level of sarcasm as 1 out of 5.


In [485]:
#Sentiment

text="Peaches from Georgia are the okay!"

prompt = f"Giving the following text, return only a valid JSON object of sentiment analysis. skip the preamble text: {text} "
print(model_invoke(prompt))

{"sentiment": "positive"}


In [486]:
# Question and Answering
question = ["What is Amazon Bedrock?","How many models does Amazon Bedrock offer?", " Does Amazon Bedrock support RAG?", "who is brad pitt?"]	

for i in question:
    prompt = f"Given the following text, answer the question. If the answer is not in the text, 'say you do not know': {i} text: {text} "
    print(f"question: {i}")
    print(f"Response : {model_invoke(prompt)} \n")	

question: What is Amazon Bedrock?
Response : Unfortunately, the text does not mention anything about "Amazon Bedrock". I do not know what Amazon Bedrock is based on the given text. The text talks about peaches from Georgia, but does not provide any information to answer the question about Amazon Bedrock. 

question: How many models does Amazon Bedrock offer?
Response : Unfortunately, the given text does not provide enough information to determine how many models Amazon Bedrock offers. The text talks about peaches from Georgia, but does not mention Amazon Bedrock or any models it might offer. So I do not know how many models Amazon Bedrock offers based on this text. 

question:  Does Amazon Bedrock support RAG?
Response : Unfortunately, the given text does not provide enough information to determine whether Amazon Bedrock supports RAG. The text talks about peaches from Georgia, but does not mention Amazon Bedrock or RAG. So I do not know if Amazon Bedrock supports RAG based on this text

## AI21 Labs Jurassic ---  ** No models available on Bedrock for AI21 Labs **

**The code below returns error message because Bedrock does not have an AI21 Labs models available.**

In [350]:
def invoke_jurassic_model(bedrock_runtime, prompt):
    # Define the request body
    
    #model_id= "ai21.j2-grande-instruct" #this one does not work
    model_id= "ai21.j2-mid-v1"
    
    max_tokens=100
    temperature=0.3
    
    
    body = {
        "prompt": prompt,
        "maxTokens": max_tokens,
        "temperature": temperature
    }

    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"

    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(body), 
        modelId=model_id, 
        accept=accept, 
        contentType=content_type
    )

    # Parse the response body
    response_body = json.loads(response["body"].read().decode('utf-8'))

    return response_body

In [351]:
prompt = "Write a short story about a robot learning to paint."

response = invoke_jurassic_model(bedrock_runtime, prompt)
print(response)

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeModel operation: You don't have access to the model with the specified model ID.

## Amazon Titan

In [235]:
import json

def invoke_titan_model2(bedrock_runtime, prompt, max_tokens=100, temperature=0.1):
   
    # Define the request body
    model_id="amazon.titan-text-lite-v1"
        
    body = {
        "inputText": prompt
    }
    
    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"
    
    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(body), 
        modelId=model_id,  
        accept=accept, 
        contentType=content_type
    )
    
    # Parse the response body
    response_body = json.loads(response["body"].read().decode('utf-8'))
    
    return response_body

In [236]:
prompt = "what is the simplest way to get a visa to australia."
response = invoke_titan_model2(bedrock_runtime, prompt)
print(response['results'][0]['outputText'])

print()
print()

question = ["What is Amazon Bedrock?","How many models does Amazon Bedrock offer?", " Does Amazon Bedrock support RAG?", "who is brad pitt?"]	

for i in question:
    prompt = f"Given the following text, answer the question. If the answer is not in the text, 'say you do not know': {i} text: {text} "
    print(f"question: {i}")
    print(f"Response : {invoke_titan_model2(bedrock_runtime, i)['results'][0]['outputText']} \n")	




Here are some steps to get a visa to Australia:

1. Determine the type of visa you need.
2. Gather the required documents.
3. Complete the visa application form.
4. Pay the visa application fee.
5. Schedule an interview if required.
6. Attend your visa interview.
7. Wait for a decision on your visa application.


question: What is Amazon Bedrock?
Response : 
Amazon Bedrock is a managed service that makes foundation models from leading AI startup and Amazon's own Titan models available through APIs. The Amazon Bedrock service makes it easy to get started with AI. You can get started for free with Amazon Bedrock and connect your existing tools and favorite Amazon Web Services (AWS) tools.  

question: How many models does Amazon Bedrock offer?
Response : 
Amazon Bedrock is a managed service that makes foundation models from leading AI startup and Amazon's own Titan models available through APIs. You can visit the bedrock product page to learn more. 

question:  Does Amazon Bedrock suppo

In [226]:
text = "Peaches from Georgia are the best in the country!"

prompt = f"Giving the following text, return the result of the sentiment analysis formetted as a json file. skip the preamble text: {text} "
print(invoke_titan_model2(bedrock_runtime, prompt)['results'][0]['outputText'])

Here is the sentiment analysis of the provided text:

```
{
    "sentiment": "positive",
    "comparative": "best"
}
```


## Cohere - Command

In [308]:
import json

def invoke_cohere_model(bedrock_runtime, prompt):
    
    model_id="cohere.command-light-text-v14"  #This model works with the payload configuration below.

    ## Tried with some of the models below, most of them gave me an error message
    
    #cohere.command-r-plus-v1:0
    #cohere.command-text-v14
    #cohere.command-light-text-v14
    #cohere.command-r-plus-v1:0
    #cohere.command-r-v1:0
    
    # Setting temperature at 0.9 to allow the model to be more "creative"
    payload = {
        "prompt": prompt,
        "max_tokens": 512,
        "temperature": 0.9
    }
    
    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"
    
    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(payload), 
        modelId=model_id,
        accept=accept, 
        contentType=content_type
    )
    
    # Parse the response body
    response_body = json.loads(response["body"].read().decode('utf-8'))
    
    return response_body



In [309]:
prompt = "Rewrite the Red Riding Hood story but having the wolf being the good guy. The story should only be a summary."

response = invoke_cohere_model(bedrock_runtime, prompt)

print(response['generations'][0]['text'])

 Wolf in sheep's clothing:

The girl in green
Came across the valley
Had to traverse it in the dark
She did not want to get murdered like last time
So she opted for the risky journey on foot
Under the full moon, she saw lights, quite bright
She slowed down, and then she heard:

Ding dong!
Who is there?
A kindly voice called out
“Please open the door, I am no danger, don’t fear!”

So, some smarts and courage, the girl opened the door, and what a surprise!
It was the wolf in the sheep’s skin, or so it seemed.

A warm welcome, and tea prepared,
The story she told, of a “wicked” family member.
The old fox ready with a plan,
To prove her wrong and save the day, what joy!

So, the moral of the story is this:
Look carefully, and you may find:
The good in the bad, and the bad in the good
Be they clothes, or people, it’s hard to tell!


## Llama

In [337]:
import json

def invoke_llama3_model(bedrock_runtime, prompt):
    
    model_id="meta.llama3-70b-instruct-v1:0"  
    
    ## Both of these models work
    
    #meta.llama3-70b-instruct-v1:0
    #meta.llama3-8b-instruct-v1:0
    
    formatted_prompt = f"""
    <|begin_of_text|><|start_header_id|>user<|end_header_id|>
    {prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """
    
    # Setting temperature at 0.9 to allow the model to be more "creative"
    payload = {
        "prompt": formatted_prompt,
        "max_gen_len": 512,
        "temperature": 0.5,
    }
    
    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"
    
    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(payload), 
        modelId=model_id,
        accept=accept, 
        contentType=content_type
    )
    
    # Parse the response body
    response_body = json.loads(response["body"].read().decode('utf-8'))
    
    return response_body

In [338]:
prompt = "Rewrite the Red Riding Hood story but having the wolf being the good guy. The story should only be a summary."

response = invoke_llama3_model(bedrock_runtime, prompt)

print(response['generation'])

 Here is a rewritten summary of Red Riding Hood where the wolf is the good guy:

In this version of the classic tale, the wolf is not a villain, but a hero who saves the day. Little Red Riding Hood, a kind and gentle soul, is on her way to visit her sick grandmother in the woods. However, the woods are fraught with danger, and a wicked woodsman has been terrorizing the forest, stealing food from the other animals and threatening their safety.

The wolf, who has been watching over the forest and its inhabitants, discovers the woodsman's evil plans and decides to intervene. He cleverly distracts the woodsman, allowing Little Red Riding Hood to safely reach her grandmother's house.

When the woodsman tries to attack Grandmother's house, the wolf bravely chases him away, saving the day and earning the gratitude of Little Red Riding Hood and her grandmother. The wolf's bravery and kindness are rewarded, and he becomes a beloved member of the forest community, celebrated for his heroism and 

In [339]:
prompt = "Can you give me 2 real life applications of RAG systems."

response = invoke_llama3_model(bedrock_runtime, prompt)

print(response['generation'])

 Here are two real-life applications of RAG (Resource Allocation and Gantt Chart) systems:

**1. Construction Project Management**

A construction company is building a new high-rise office building in a city center. The project involves multiple tasks, such as excavation, foundation work, structural construction, electrical and plumbing installation, and finishing work (e.g., drywall, flooring, and painting). The project manager uses a RAG system to plan and schedule the tasks, allocate resources (e.g., labor, equipment, and materials), and track progress.

The RAG system helps the project manager to:

* Identify dependencies between tasks and create a logical sequence of activities
* Allocate resources to each task, ensuring that the right personnel and equipment are assigned to each activity
* Develop a Gantt chart to visualize the project schedule and identify potential bottlenecks and delays
* Monitor progress and adjust the schedule as needed to ensure the project is completed on

## Mistral

In [335]:
import json

def invoke_mistral_model(bedrock_runtime, prompt):
    
    model_id="mistral.mistral-large-2402-v1:0"
    
    #amazon.titan-text-lite-v1
    #cohere.command-r-v1:0
    
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"
    
    # Define the request payload
    payload = {
        "prompt": formatted_prompt,
        "max_tokens": 512,
        "temperature": 0.1,
    }
    
    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"
    
    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(payload), 
        modelId=model_id,
        accept=accept, 
        contentType=content_type
    )
    
    # Parse the response body
    response_body = json.loads(response["body"].read().decode('utf-8'))
    
    return response_body

In [336]:
prompt = "Rewrite the Red Riding Hood story but having the wolf being the good guy. The story should only be a summary."

prompt = "Write a function in python that gets an array of numbers and an integer number as parameters and returns a matrix of n rows and m columns where n is the lenght of the elements in the array and m is the integer number.  The cells should be calculated as the multiplication from each array member by the integer given as parameter"


response = invoke_mistral_model(bedrock_runtime, prompt)

print(response["outputs"][0]["text"])

 Sure, here is a Python function that should do what you're asking for:

```python
def create_matrix(array, integer):
    # Get the length of the array
    n = len(array)
    # Create an empty matrix with n rows and m columns
    matrix = [[0] * integer for _ in range(n)]
    # Fill the matrix with the multiplication of each array member by the integer
    for i in range(n):
        for j in range(integer):
            matrix[i][j] = array[i] * integer
    # Return the matrix
    return matrix
```

You can use this function like this:

```python
numbers = [1, 2, 3, 4, 5]
integer = 3
print(create_matrix(numbers, integer))
```

This will output:

```python
[[3, 3, 3], [6, 6, 6], [9, 9, 9], [12, 12, 12], [15, 15, 15]]
```

Each row of the matrix is the corresponding number from the array multiplied by the integer.


## Stability AI - Stable Diffusion

In [476]:
import os
import random
import json
import base64

def invoke_Stable_difusion_model(bedrock_runtime, prompt):
    
    model_id="stability.stable-diffusion-xl-v1"
    
    # Generate a random seed.
    seed = random.randint(0, 4294967295)
    
    # Define the request payload
    payload = {
        "text_prompts": [{"text": prompt}],
        "style_preset": "photographic",
        "seed": seed,
        "cfg_scale": 10,
        "steps": 30,
    }
    
    # Define the accept and content type headers
    accept = "application/json"
    content_type = "application/json"
    
    # Invoke the model using bedrock_runtime
    response = bedrock_runtime.invoke_model(
        body=json.dumps(payload), 
        modelId=model_id,
        accept=accept, 
        contentType=content_type
    )
    
    # Parse the response body
    response_body = json.loads(response["body"].read())
    
    return response_body

In [480]:
#prompt = "A pink monkey riding a camel in the city"
prompt = "A cartoon soda can walking on an busy street, holding a sword in his hand"


response = invoke_Stable_difusion_model(bedrock_runtime, prompt)

base64_image_data = response["artifacts"][0]["base64"]

#print(response["artifacts"][0]["base64"])

# Save the generated image to a local folder.
i, output_dir = 1, "output"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
while os.path.exists(os.path.join(output_dir, f"stability_{i}.png")):
    i += 1

image_data = base64.b64decode(base64_image_data)

image_path = os.path.join(output_dir, f"result_{i}.png")
with open(image_path, "wb") as file:
    file.write(image_data)

print(f"The generated image has been saved to {image_path}")

The generated image has been saved to output/result_2.png


## --------------------------

# SECTION 2:  Invoking models - (Invoke model with response stream)

## Amazon Titan

In [395]:
from IPython.display import clear_output, display, display_markdown, Markdown

import time

prompt = "Describe what is the most efficient way to learn a new language."

modelId = "amazon.titan-tg1-large"

#Both of these 2 models work.

#amazon.titan-text-lite-v1
#amazon.titan-tg1-large

#configuring the payload

payload = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.9,
    },
}

body = json.dumps(payload)

#accept = "application/json"
#contentType = "application/json"

response = bedrock_runtime.invoke_model_with_response_stream(
    body=body, modelId=modelId
)


# Extract and print the response text in real-time.
for event in response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if "outputText" in chunk:
        print(chunk["outputText"], end="")
        



Learning a new language requires time, dedication, and practice. One of the most efficient ways to learn a language is to immerse yourself in the culture and language. This can involve traveling to a country where the language is spoken, watching movies and TV shows in that language, or even finding language exchange partners. 

Another effective method is to use language learning apps, such as Duolingo or Babbel. These apps provide interactive lessons and exercises that help you learn vocabulary, grammar, and sentence structure. You can also use language learning podcasts, such as Coffee Break Spanish or Pimsleur. 

Reading books, newspapers, and magazines in the target language is also a great way to improve your language skills. You can start by reading simple books and gradually work your way up to more complex texts. 

In addition to these methods, practicing with a language partner can be extremely beneficial. You can practice speaking, listening, and writing with a native speak

In [ ]:
"""
###  This code also presents the output on a stream.

#stream = response.get('body')
#output = []

#if stream:
#    for event in stream:
#        chunk = event.get('chunk')
#        if chunk:
#            chunk_obj = json.loads(chunk.get('bytes').decode())
#            text = chunk_obj['outputText']
#            clear_output(wait=True)
#            output.append(text)
#            display_markdown(Markdown(''.join(output)))
         
"""

## Antropic - claude

In [469]:

  
#bedrock = boto3.client(service_name='bedrock-runtime')
prompt = "Can you give me 2 real life applications of RAG systems."
prompt = "Can you give me summary of how neural networks are trainned. How the weights are calculated and the backpropagation process."

    
model_Id="anthropic.claude-3-haiku-20240307-v1:0"
    #anthropic.claude-3-haiku-20240307-v1:0
    #anthropic.claude-3-sonnet-20240229-v1:0


request_body = {
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "temperature":0.2,
    "messages": [
      {
          "role": "user",
          "content": [{"type": "text", "text": prompt}],
      }
    ], 
    }
body = json.dumps(request_body)

    
# Invoke the Anthropic Claude Sonnet model
response = bedrock_runtime.invoke_model_with_response_stream(
    modelId=model_Id,
    body=body
)

# Extract and print the response text in real-time.
for event in response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if chunk["type"] == "content_block_delta":
        print(chunk["delta"].get("text", ""), end="")
   

Certainly! Neural networks are trained using a process called backpropagation, which involves adjusting the weights of the connections between the neurons in the network to minimize the error between the network's output and the desired output.

Here's a summary of how neural networks are trained:

1. **Initialization**: The weights of the connections between the neurons in the network are initialized to small random values. This is done to break the symmetry and ensure that the network can learn different features.

2. **Forward Propagation**: The input data is fed into the network, and the output is calculated by propagating the input through the network layer by layer. This is called the forward propagation step.

3. **Error Calculation**: The output of the network is compared to the desired output (the ground truth), and the error is calculated using a loss function, such as mean squared error or cross-entropy.

4. **Backpropagation**: The error is then propagated backwards through

## Cohere - Command

In [470]:
import time

# Set the model ID, e.g., Command R.
model_id = "cohere.command-light-text-v14"

# Define the prompt for the model.
prompt = "write a summary of how the second world war started."

# Format the request payload using the model's native structure.
payload = {
    "prompt": prompt,
    "max_tokens": 2000,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(payload)

# Invoke the model with the request.
streaming_response = bedrock_runtime.invoke_model_with_response_stream(
    modelId=model_id, body=request
)

# Extract and print the response text in real-time.
for event in streaming_response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if "generations" in chunk:
        print(chunk["generations"][0]["text"], end="")


 The Second World War started in September 1939, when Germany, under Chancellor Adolf Hitler, invaded Poland from the west and from the east together with their allied partners, the Soviet Union, led by Joseph Stalin. This was the final step of their plan to occupy the country and eliminate its capital, Warsaw, from the map. This attack started the whole war, as Great Britain, which had been defied by Hitler since the rise to power in Germany, declared war on Germany after the latter’s failure to respond to the demand to abstain from further aggression. The Soviet Union, seeing the danger, invaded from the east, and the German army could not stop them.

Later in the year, Germany also attacked Denmark and Norway, which were easily conquered due to their neutrality and good relationships with the Axis powers, especially Germany. After that, Germany began its campaign in the East, against Russia, and by May 1940, the Germans had advanced deep into the Soviet Union, reaching the River bou

## Llama

In [475]:

model_id = "meta.llama3-1-8b-instruct-v1:0"

# Both of these models work. 

#meta.llama3-1-8b-instruct-v1:0
#meta.llama3-70b-instruct-v1:0

# Define the prompt for the model.
prompt = "Describe how the activation functions in a neural network work. Use language suitable for an executive with little technical knowledge."

formatted_prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

# Format the request payload using the model's native structure.
payload = {
    "prompt": formatted_prompt,
    "max_gen_len": 512,
    "temperature": 0.0,
}

body = json.dumps(payload)

response = bedrock_runtime.invoke_model_with_response_stream(
    modelId=model_id,
    body=body
)

for event in response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if "generation" in chunk:
        print(chunk["generation"], end="")

Imagine you're running a restaurant, and you have a team of employees who take orders, prepare food, and serve customers. Each employee has a specific role, and they work together to get the job done.

In a neural network, the "employees" are called "neurons" or "nodes." They receive information, process it, and then send it to the next employee (node) in the chain. The "orders" they receive are called "inputs," and the "food" they produce is called the "output."

The key to a neural network is the "activation function," which is like the employee's "recipe book." It tells each employee (node) how to process the information they receive and what to do with it.

Think of the activation function like a simple decision tree:

1. **Input**: The employee receives an order (input) from the previous node.
2. **Processing**: The employee looks at the order and decides what to do with it based on their recipe book (activation function).
3. **Output**: The employee produces a new order (output) 

## Mistral

In [481]:

# Set the model ID, e.g., Mistral Large.
model_id = "mistral.mistral-large-2402-v1:0"

# Define the prompt for the model.
prompt = "Give me the steps to apply for Canadien residency being a Guatemalan citizen."

# Embed the prompt in Mistral's instruction format.
formatted_prompt = f"<s>[INST] {prompt} [/INST]"

# Formatting the payload for this model

payload = {
    "prompt": formatted_prompt,
    "max_tokens": 512,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(payload)

streaming_response = bedrock_runtime.invoke_model_with_response_stream(
        modelId=model_id, body=request
    )

for event in streaming_response["body"]:
    chunk = json.loads(event["chunk"]["bytes"])
    if "outputs" in chunk:
        print(chunk["outputs"][0].get("text"), end="")

 Sure, here are the general steps to apply for Canadian permanent residency as a Guatemalan citizen. Please note that the process can be complex and time-consuming, and it's always a good idea to consult with a Canadian immigration lawyer or consultant for personalized advice.

1. **Determine your eligibility**: The first step is to determine if you're eligible to apply for Canadian permanent residency. There are several programs through which you can apply, including Express Entry, Provincial Nominee Programs, Quebec-selected skilled workers, and others. Each program has its own eligibility criteria.

2. **Language Test**: You will need to prove your proficiency in either English or French. This usually involves taking a standardized test such as the IELTS for English or the TEF for French.

3. **Education Credential Assessment (ECA)**: If you're applying through Express Entry, you'll likely need an ECA to show that your foreign education is equivalent to Canadian education standards.

## Stability AI - Stable Diffusion is a model generally used for image generation and not text stream. Therefore, is not applicable for this section.